In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dropout
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import Model
import os


In [2]:
# These values are arbitrarily chosen until network is finalised
batch_size = 16
depth = 64
categories = 2
epochs = 5
n = 128
m = 128

In [3]:
def build_improved_unets(input_shape, depth):
    inputs = Input(input_shape)
    
    conv1 = Conv2D(filters=depth, kernel_size=(3,3), padding="same", activation="relu", kernel_initializer="he_normal")(inputs)
    conv1 = Conv2D(filters=depth, kernel_size=(3,3), padding="same", activation="relu", kernel_initializer="he_normal")(conv1)
    drop1 = Dropout(conv1)
    
    model = Model(inputs, conv1)
    model.summary()
    
    return model

In [4]:
model = build_improved_unets(input_shape=(n,m,2), depth=depth)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 2)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 64)      1216      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      36928     
Total params: 38,144
Trainable params: 38,144
Non-trainable params: 0
_________________________________________________________________


In [5]:
def process_images(path, segmentation):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        directory = path, 
        labels=None,
        label_mode = 'binary',
        batch_size = batch_size,
        validation_split = 0.2,
        subset=segmentation,
        image_size = (n,m),
        color_mode = 'rgb',
        shuffle = True,
        seed = 45820188
    )
    
    return dataset

In [11]:
X_train_ds = process_images("C:\ISIC Dataset\Smaller\Train", "training")
y_train_ds = process_images("C:\ISIC Dataset\Smaller\Seg", "training")

X_test_ds = process_images("C:\ISIC Dataset\Smaller\Train", "validation")
y_test_ds = process_images("C:\ISIC Dataset\Smaller\Seg", "validation")

# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 10))
# for images in X_train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.axis("off")
        


Found 13 files belonging to 1 classes.
Using 11 files for training.
Found 13 files belonging to 1 classes.
Using 11 files for training.
Found 13 files belonging to 1 classes.
Using 2 files for validation.
Found 13 files belonging to 1 classes.
Using 2 files for validation.


In [13]:
import tensorflow_datasets as tfds

X_train = None
for image in tfds.as_numpy(X_train_ds):
    X_train = image
    
y_train = None
for image in tfds.as_numpy(y_train_ds):
    y_train = image
    
X_test = None
for image in tfds.as_numpy(X_test_ds):
    X_test = image
    
y_test = None
for image in tfds.as_numpy(y_test_ds):
    y_test = image

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(X_test, y_test))

Epoch 1/5


ValueError: in user code:

    C:\Users\Andrew\anaconda3\envs\s4582018\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\Andrew\anaconda3\envs\s4582018\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Andrew\anaconda3\envs\s4582018\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Andrew\anaconda3\envs\s4582018\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Andrew\anaconda3\envs\s4582018\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Andrew\anaconda3\envs\s4582018\lib\site-packages\keras\e